# Серия тиков с разной ценой, количество тиков N, расстояние во времени меньше T миллисекунд

In [1]:
from simple.ohlc import debounce, midPrice, npJoin
from simple.plotly import chartFigure
from simple.funcs import symlog
from simple.backtest import npBacktestMarket, getProfit
import numpy as np
from simple.numpy import rolling_apply

In [2]:
threshold = 2_000_000  # default resolution in microseconds
period = 30

In [3]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)[:30000]
T[-5:]

rec.array([('2022-05-08T00:19:22.420000', 29995, 2629.81,    -5.),
           ('2022-05-08T00:19:22.420000', 29996, 2629.81,    -4.),
           ('2022-05-08T00:19:22.420000', 29997, 2629.8 , -1140.),
           ('2022-05-08T00:19:22.420000', 29998, 2629.8 ,  -100.),
           ('2022-05-08T00:19:22.420000', 29999, 2629.78,    -3.)],
          dtype={'names':['DateTimeA','LocalTimeA','PriceA','VolumeA'], 'formats':['<M8[us]','<i8','<f8','<f8'], 'offsets':[0,8,16,24], 'itemsize':40})

In [4]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]

In [5]:
C = debounce(T)
C[-5:]

rec.array([('2022-05-08T00:19:22.420000', 3564, 2629.915, 1000, 1.9750e+03, 0., -1.9750e+03,  2, 0,  2),
           ('2022-05-08T00:19:22.420000', 3565, 2629.885,    0, 3.7000e+01, 0., -3.7000e+01,  1, 0,  1),
           ('2022-05-08T00:19:22.420000', 3566, 2629.855,    0, 6.5350e+03, 0., -6.5350e+03,  6, 0,  6),
           ('2022-05-08T00:19:22.420000', 3567, 2629.825,    0, 1.5953e+04, 0., -1.5953e+04, 10, 0, 10),
           ('2022-05-08T00:19:22.420000', 3568, 2629.785,    0, 3.0000e+00, 0., -3.0000e+00,  1, 0,  1)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [6]:
len(T), len(C)

(30000, 3569)

In [7]:
C.Volume = C.BuySize + C.SellSize
Buy = C[C.Volume > 0]
Sell = C[C.Volume < 0]
Sell[-5:]

rec.array([('2022-05-08T00:19:22.420000', 3564, 2629.915, 1000, -1.9750e+03, 0., -1.9750e+03,  2, 0,  2),
           ('2022-05-08T00:19:22.420000', 3565, 2629.885,    0, -3.7000e+01, 0., -3.7000e+01,  1, 0,  1),
           ('2022-05-08T00:19:22.420000', 3566, 2629.855,    0, -6.5350e+03, 0., -6.5350e+03,  6, 0,  6),
           ('2022-05-08T00:19:22.420000', 3567, 2629.825,    0, -1.5953e+04, 0., -1.5953e+04, 10, 0, 10),
           ('2022-05-08T00:19:22.420000', 3568, 2629.785,    0, -3.0000e+00, 0., -3.0000e+00,  1, 0,  1)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [8]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names':['DT','A','B'], 'formats':['<M8[us]','<f4','<f4'], 'offsets':[0,8,16], 'itemsize':24})

In [9]:
K = npJoin(C.DT, S.DT)
len(K)

3569

In [10]:
# calculate sum of duration/volume by sliding window
def getSum(x):
    return np.sum(x).astype(int)


DurationA = rolling_apply(getSum, period, C.Duration)
VolumeA = rolling_apply(getSum, period, C.Volume)
len(DurationA)

3569

In [11]:
# trade signal
signal = 0
SignalA = np.zeros_like(DurationA)
for i in range(len(SignalA)):
    if DurationA[i] < threshold and VolumeA[i] > 0:
        signal = 1
    elif DurationA[i] < threshold and C.Volume[i] < 0:
        signal = -1
    SignalA[i] = signal
        
SignalA[-10:]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [16]:
D = npBacktestMarket(S.DT[K], S.A[K], S.B[K], SignalA, 0)
D[-5:]

rec.array([(3103, '2022-05-08T00:17:07.656000', 2634.35009766, 2634.39501953, 3366, '2022-05-08T00:19:00.114000', 2632.39990234, 2632.39501953, -1.),
           (3366, '2022-05-08T00:19:00.114000', 2632.39990234, 2632.39501953, 3382, '2022-05-08T00:19:01.094000', 2632.01000977, 2632.05004883,  1.),
           (3382, '2022-05-08T00:19:01.094000', 2632.01000977, 2632.05004883, 3395, '2022-05-08T00:19:02.182000', 2632.15991211, 2632.15478516, -1.),
           (3396, '2022-05-08T00:19:03.889000', 2632.15991211, 2632.14990234, 3493, '2022-05-08T00:19:17.753000', 2631.0300293 , 2631.03515625,  1.),
           (3493, '2022-05-08T00:19:17.753000', 2631.0300293 , 2631.03515625, 3527, '2022-05-08T00:19:18.307000', 2630.60009766, 2630.57006836, -1.)],
          dtype=[('X0', '<i8'), ('T0', '<M8[us]'), ('Price0', '<f8'), ('MidPrice0', '<f8'), ('X1', '<i8'), ('T1', '<M8[us]'), ('Price1', '<f8'), ('MidPrice1', '<f8'), ('Size', '<f8')])

In [17]:
Long = D[D.Size > 0]
Short = D[D.Size < 0]

P = getProfit(D)
P[-5:]

rec.array([(3366, '2022-05-08T00:19:00.114000',  1.95019531,  2.        , 2.633375  , -0.68317969),
           (3382, '2022-05-08T00:19:01.094000', -0.38989258, -0.3449707 , 2.63220496, -3.02209753),
           (3395, '2022-05-08T00:19:02.182000', -0.14990234, -0.10473633, 2.63208496, -2.7819873 ),
           (3493, '2022-05-08T00:19:17.753000', -1.12988281, -1.11474609, 2.63159497, -3.76147778),
           (3527, '2022-05-08T00:19:18.307000',  0.42993164,  0.46508789, 2.63081506, -2.20088342)],
          dtype=[('Index', '<i8'), ('DT', '<M8[us]'), ('RawPnL', '<f8'), ('MidPnL', '<f8'), ('Fee', '<f8'), ('Profit', '<f8')])

In [21]:
# indexed chart with bidask
fig = chartFigure(height=800, rows=4,
    Sell=dict(x=Sell.Index, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Volume)-1, opacity=0.25),
    Buy=dict(x=Buy.Index, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Volume)-1, opacity=0.25),
    Bid=dict(x=C.Index, y=S.B[K], color='green', opacity=0.2),
    Ask=dict(x=C.Index, y=S.A[K], color='red', opacity=0.2),
                  
    # trades 
    EnterLong=dict(x=Long.X0, y=Long.Price0, mode='markers',
                   marker=dict(symbol='triangle-up', size=12, line_color='darkgreen', line_width=1, color='green')),
    ExitLong=dict(x=Long.X1, y=Long.Price1, mode='markers',
                  marker=dict(symbol='x', size=10, line_color='darkgreen', line_width=1, color='green', opacity=0.4)),

    EnterShort=dict(x=Short.X0, y=Short.Price0, mode='markers',
                    marker=dict(symbol='triangle-down', size=12, line_color='darkred', line_width=1, color='red')),
    ExitShort=dict(x=Short.X1, y=Short.Price1, mode='markers',
                   marker=dict(symbol='x', size=10, line_color='darkred', line_width=1, color='red', opacity=0.4)),
            
    # Signal and duration indicators              
    Signal=dict(x=C.Index, y=SignalA, color='magenta', row=3, shape='hv'),
    Duration=dict(x=C.Index, y=DurationA, color='blue', row=4),
                  
    # Profit              
    RawPnL=dict(x=P.Index, y=P.RawPnL.cumsum(), secondary_y=True, width=4, color='gray', opacity=0.2, shape='hv', connectgaps=True),
    Profit = dict(x=P.Index, y=P.Profit.cumsum(), secondary_y=True, width=6, color='blue', opacity=0.4, shape='hv', connectgaps=True)
)

fig.add_bar(name='Buy', x=Buy.Index, y=Buy.Volume, row=2, col=1, marker_color='green')
fig.add_bar(name='Sell', x=Sell.Index, y=Sell.Volume, row=2, col=1, marker_color='red')
fig.add_hline(y=threshold, row=4, col=1, line_dash='dot', line_color='gray')

# styling
fig.update_xaxes(spikemode='across+marker', spikedash='dot', spikethickness=2, spikesnap='cursor')
fig.update_traces(xaxis=f'x4')
fig.update_layout(legend=dict(x=0, y=1.05, bordercolor="lightgray", borderwidth=1))

fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.25,
   …